In [1]:
def cleanup(desc):
  desc = desc.replace('<|startoftext|>','').replace('<|endoftext|>\n','')
  return desc.strip()

In [2]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/labs15-pain-point/Data-Science/master/generated/log(6).csv')
df = df.drop('Unnamed: 0', axis=1)
df['descriptions'] = df['descriptions'].apply(cleanup)
df = df[df['descriptions'] != '']
df.head()

,descriptions
0,"Founded in 2017, Radiantia is a healthcare tec..."
1,Rumble Wireless is a fabless mobility company ...
2,Rent the Mile is a marketplace that helps make...
3,ReThink Medical is a medical device research c...
4,Ritual is a fashion ordering and optimization ...


In [3]:
# Function to make the output more readable
def print_line(line):
  while len(str(line)) > 90:
    print(line[:90])
    line = line[90:]
  print(line,'\n\n')

In [4]:
def word_freq(line):
  line = str(line).lower()
  full_len = len(line.split())
  dump_words = ['founded by', 'is based in', 'was founded in', 
                'is headquartered in', 'headquarters in', 'developed by',
                'developed in', 'additional offices', 'germany', 'usa',
                'france', 'uk', 'china', 'california', 'india', 
                'silicon valley', 'san francisco', 'established in',
                'mountain view', 'family owned', 'family-owned', 
                'clients include', 'argentina', 'brazil', 'chile', 'colombia', 
                'japan', 'korea', 'malaysia', 'mexico']
  
  # Drop words that encode garbage info for our purposes
  for phrase in dump_words:
    line = line.replace(phrase,'')
  
  # Calculate unique word frequency
  split_line = str(line).split()
  if len(split_line) < 5: return 0
  return len(set(split_line)) / full_len

In [5]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()


def entity_freq(text, nlp):
  doc = nlp(text)
  count = 0
  for X in doc.ents:
    if (X.label_ in ['ORG', 'DATE', 'PERSON', 'TIME', 'PERCENT', 'MONEY']):
      count += 1
  return count/len(text.split())

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

real_desc = pd.read_csv('big_boy_df/big_boy_df.csv')
real_desc = real_desc[real_desc['Full Description'].notnull()]['Full Description']

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(real_desc)

# Instantiate the model
nn = NearestNeighbors(n_neighbors=2, algorithm='ball_tree')

# Fit on TF-IDF Vectors
nn.fit(dtm.todense())

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=2, p=2, radius=1.0)

In [13]:
def get_distance(desc, nn_model):
  new = tfidf.transform([desc])
  neighborhood = nn.kneighbors(new.todense())
  return neighborhood[0][0][0]

In [ ]:
df['freq'] = df['descriptions'].apply(word_freq)
df = df[(df['freq'] > .6)].reset_index()
df['ent_freq'] = [entity_freq(desc, nlp) for desc in df['descriptions']]
df['dist_to_real'] = [get_distance(desc, nn) for desc in df['descriptions']]

In [ ]:
df.head()

In [ ]:
df.describe()